In [1]:
import sionna.rt
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from tqdm import tqdm
%matplotlib inline
import os
import sys
from sionna.rt import load_scene, AntennaArray, PlanarArray, Transmitter, Receiver, Camera,\
                      PathSolver, RadioMapSolver, subcarrier_frequencies

# Add the parent directory of 'examples' to the module search path (to get utils in src)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils import  load_config
from src.ue_locations_generator import create_user_location_dataset
import logging
import datetime
print("Sionna version:", sionna.rt.__version__)
print("tf version:", tf.__version__)

2025-06-19 09:12:41.836328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750324361.857885  675943 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750324361.864529  675943 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750324361.882517  675943 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750324361.882542  675943 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750324361.882545  675943 computation_placer.cc:177] computation placer alr

Sionna version: 1.0.2
tf version: 2.19.0


In [2]:
gpus = tf.config.list_physical_devices('GPU')
print("GPU:", gpus)

if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU') # only use the first GPU
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    print("Using GPU:", gpus[0].name)
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

else:
  print("No GPU found, using CPU.")

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPU
Using GPU: /physical_device:GPU:0


I0000 00:00:1750324364.612718  675943 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13741 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
# load config file
config = load_config()
# create user dataset
dataset_path, ue_dataset_path = create_user_location_dataset(config)
# load all ue positions
ds_users = xr.load_dataset(ue_dataset_path)
# load scene# Load scene
scene = load_scene(config['paths']['scenepath']) 

stripe 0, ru: 4, pos: [2.0, 4.5, 1.5] - point outside of zone: True
stripe 0, ru: 5, pos: [2.0, 5.0, 1.5] - point outside of zone: True
stripe 0, ru: 6, pos: [2.0, 5.5, 1.5] - point outside of zone: True
stripe 0, ru: 24, pos: [2.0, 14.5, 1.5] - point outside of zone: True
stripe 0, ru: 25, pos: [2.0, 15.0, 1.5] - point outside of zone: True
stripe 0, ru: 26, pos: [2.0, 15.5, 1.5] - point outside of zone: True
stripe 1, ru: 4, pos: [2.5, 4.5, 1.5] - point outside of zone: True
stripe 1, ru: 5, pos: [2.5, 5.0, 1.5] - point outside of zone: True
stripe 1, ru: 6, pos: [2.5, 5.5, 1.5] - point outside of zone: True
stripe 1, ru: 24, pos: [2.5, 14.5, 1.5] - point outside of zone: True
stripe 1, ru: 25, pos: [2.5, 15.0, 1.5] - point outside of zone: True
stripe 1, ru: 26, pos: [2.5, 15.5, 1.5] - point outside of zone: True
stripe 2, ru: 4, pos: [3.0, 4.5, 1.5] - point outside of zone: True
stripe 2, ru: 5, pos: [3.0, 5.0, 1.5] - point outside of zone: True
stripe 2, ru: 6, pos: [3.0, 5.5, 1.5

2025-06-19 09:12:45 WARN wrk0 [HDRFilm] Monochrome mode enabled, setting film output pixel format to 'luminance' (was rgb).

In [4]:
# configure tx and rx arrays
N_antennas = config['antenna_config']['N_antennas_per_axis']
print(f'number antennas per axis: {N_antennas}')

scene.tx_array = PlanarArray(num_rows=N_antennas,
                            num_cols=N_antennas,
                            vertical_spacing=0.5,
                            horizontal_spacing=0.5,
                            pattern=config['antenna_config']['pattern'],
                            polarization=config['antenna_config']['polarization'])

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=N_antennas,
                            num_cols=N_antennas,
                            vertical_spacing=0.5,
                            horizontal_spacing=0.5,
                            pattern=config['antenna_config']['pattern'],
                            polarization=config['antenna_config']['polarization'])


# sub-THz stripe specs 
stripe_start_pos = config['stripe_config']['stripe_start_pos']
N_RUs = config['stripe_config']['N_RUs'] # adjust to size of the room (along y axis)
N_stripes = config['stripe_config']['N_stripes']# adjust to size of the room (alang x axis)
total_N_RUs = N_RUs * N_stripes # total number of radio units
space_between_RUs = config['stripe_config']['space_between_RUs'] # in meters
space_between_stripses = config['stripe_config']['space_between_stripes'] # in meters

number antennas per axis: 2


In [5]:
# add users
# loop over al ue postions
for ue_idx in range(ds_users.dims['user']):

    # get coordinates
    x, y, z = ds_users.x.values[ue_idx], ds_users.y.values[ue_idx], ds_users.z.values[ue_idx]
    ue_pos = [float(x), float(y), float(z)]

    # Create a receiver
    rx = Receiver(name=f"rx_{ue_idx}",
                position=ue_pos,
                display_radius=0.1,
                color=(1, 0.937, 0.016)) #yellow
    
    # Point the receiver upwards
    rx.look_at([ue_pos[0], ue_pos[1], 3.5]) # Receiver points upwards

    # check orientation
    #print(f'rx orientation: {rx.orientation}')

    # Add receiver instance to scene
    scene.add(rx)

/tmp/ipykernel_675943/3578568748.py:3: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for ue_idx in range(ds_users.dims['user']):


In [6]:
# add stripes
# loop over all stripes
for stripe_idx in range(N_stripes):
    # loop over all RUs 
    for RU_idx in range(N_RUs):
        # compute RU position
        tx_pos = [stripe_start_pos[0] + stripe_idx * space_between_stripses,
                stripe_start_pos[1] + RU_idx * space_between_RUs,
                stripe_start_pos[2]]
        
        # Create RU transmitter instance
        tx = Transmitter(name=f"tx_stripe_{stripe_idx}_RU_{RU_idx}",
                    position=tx_pos,
                    display_radius=0.1,
                    color=[1, 0, 0])

        # Add RU transmitter instance to scene
        scene.add(tx)

        # Point the transmitter downwards
        tx.look_at([tx_pos[0], tx_pos[1], 0]) # Transmitter points downwards


In [7]:
scene.preview();